#### Which burrito trucks to select?

#### 0. Importing libraries

In [75]:
%pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import numpy as np

Note: you may need to restart the kernel to use updated packages.


#### 1. Set up the model

In [76]:
burrito_model = gp.Model()

#### 2. Define sets and parameters

In [77]:
I = [1, 2, 3, 4] # set of customers
J = [1, 2, 3, 4, 5] # set of potential truck locations

d = [50, 25, 60, 30] # demand
c = np.array([[1, 2, 3, 6, 3],
              [2, 0.5, 2, 4, 7],
              [3, 2, 1.5, 4, 5],
              [4, 3, 3, 2, 1]]) # travel distance from each customer to each truck
f = 250 # cost of setting up one truck
r = 10 # unit burrito revenue
k = 5 # unit burrito ingredient cost

#### 3. Set the variables and the objective function

In [78]:
x = burrito_model.addVars(J, vtype = GRB.BINARY, name = "x")
y = burrito_model.addVars(I, J, vtype = GRB.BINARY, name = "y")

In [79]:
y

{(1, 1): <gurobi.Var *Awaiting Model Update*>,
 (1, 2): <gurobi.Var *Awaiting Model Update*>,
 (1, 3): <gurobi.Var *Awaiting Model Update*>,
 (1, 4): <gurobi.Var *Awaiting Model Update*>,
 (1, 5): <gurobi.Var *Awaiting Model Update*>,
 (2, 1): <gurobi.Var *Awaiting Model Update*>,
 (2, 2): <gurobi.Var *Awaiting Model Update*>,
 (2, 3): <gurobi.Var *Awaiting Model Update*>,
 (2, 4): <gurobi.Var *Awaiting Model Update*>,
 (2, 5): <gurobi.Var *Awaiting Model Update*>,
 (3, 1): <gurobi.Var *Awaiting Model Update*>,
 (3, 2): <gurobi.Var *Awaiting Model Update*>,
 (3, 3): <gurobi.Var *Awaiting Model Update*>,
 (3, 4): <gurobi.Var *Awaiting Model Update*>,
 (3, 5): <gurobi.Var *Awaiting Model Update*>,
 (4, 1): <gurobi.Var *Awaiting Model Update*>,
 (4, 2): <gurobi.Var *Awaiting Model Update*>,
 (4, 3): <gurobi.Var *Awaiting Model Update*>,
 (4, 4): <gurobi.Var *Awaiting Model Update*>,
 (4, 5): <gurobi.Var *Awaiting Model Update*>}

In [80]:
burrito_model.setObjective(gp.quicksum(gp.quicksum((r - k) * d[i - 1] * y[i, j] for j in J) for i in I) - gp.quicksum(f * x[j] for j in J))

#### 4. Set the "sense" of the optimization problem

In [81]:
burrito_model.ModelSense = GRB.MAXIMIZE

#### 5. Add the constraints

In [82]:
for i in I:
    burrito_model.addConstr(gp.quicksum(y[i, j] for j in J) <= 1)

for i in I:
    for j in J:
        burrito_model.addConstr(y[i, j] <= x[j])

#### 6. Update and optimize the model

In [83]:
burrito_model.update()
burrito_model.write('burrito_truck_location.lp')
burrito_model.optimize()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 24 rows, 25 columns and 60 nonzeros
Model fingerprint: 0x92b6dc56
Variable types: 0 continuous, 25 integer (25 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+02, 3e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0000000
Presolve time: 0.00s
Presolved: 24 rows, 25 columns, 60 nonzeros
Variable types: 0 continuous, 25 integer (25 binary)
Found heuristic solution: objective 200.0000000

Root relaxation: objective 5.750000e+02, 17 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0     575.0000000  575.00000  0.00%     -    0s

Explored 1 nodes (17 simplex iterations

#### 7. Print the results

In [84]:
print(f"The optimal objective value for this problem is {burrito_model.ObjVal}") # the optimal objective value

print(f"The optimal values of the decisions variables are as follows:")
for var in burrito_model.getVars():
    if var.x >= 0:
        print(f"{var.varName} = {var.x}") # the optimal choice of decision variables

The optimal objective value for this problem is 575.0
The optimal values of the decisions variables are as follows:
x[1] = -0.0
x[2] = 1.0
x[3] = -0.0
x[4] = -0.0
x[5] = -0.0
y[1,1] = 0.0
y[1,2] = 1.0
y[1,3] = 0.0
y[1,4] = 0.0
y[1,5] = 0.0
y[2,1] = 0.0
y[2,2] = 1.0
y[2,3] = 0.0
y[2,4] = 0.0
y[2,5] = 0.0
y[3,1] = 0.0
y[3,2] = 1.0
y[3,3] = 0.0
y[3,4] = 0.0
y[3,5] = 0.0
y[4,1] = 0.0
y[4,2] = 1.0
y[4,3] = 0.0
y[4,4] = 0.0
y[4,5] = 0.0
